## Cluster-Based Summarization 分群摘要方法
Cluster-based summarization的主要概念是使用相似計算(similarity measures)把一個檔案的句子分成不同群(clusters)依照句子的相似度。因此，不同的群會代表不同的話題，然後從每群抽出得到高分的句子就能夠做成摘要。在這個示範裏，我們用的分群方法是k-means clustering，相似計算方法為cosine similarity。K-means clustering會把文檔的句子分成k群，然後每個群會各計算出一個群中心(centroid)。利用cosine similarity的相思度計算，越類似群中心的句子會得到越高的分數。最後，每群得分最高的句子會被選為做成有k句子的摘要。

Cluster-based的摘要方法的好處是可以代表多樣的句子並減少冗餘。除了單文件摘要以外，分群摘要方法也可以用來做多文件長文摘要。

## 讀取資料
- 讀取爬文和下載好的100篇文章

In [1]:
import pandas as pd
import numpy as np
import jieba
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

In [2]:
# 讀取爬下來的文集
import pandas as pd
news100 = pd.read_excel('appledaily_politinews100.xlsx')
news100.head()

,category,content,title
0,政治,總統蔡英文卸任民進黨主席，昨在臉書po給黨員的一封信，信中說，民進黨會展開一場轟轟烈烈的檢討...,蔡英文要轟轟烈烈檢討民進黨 林濁水酸：民主國家沒聽過
1,政治,民進黨這次九合一選舉慘敗，蔡英文辭去黨主席一職，昨公布「給黨員的一封信」，內容提到，最該改變...,他列民進黨10大豬隊友 「不下架將賠上2020」
2,政治,2018年地方選舉國民黨拿下空前勝利，執政地區從6縣市竄升為15縣市，昨中常會上國民黨提出選...,選戰檢討報告出爐 國民黨：落選者「缺乏對外拓票企圖心」
3,政治,準高雄市長韓國瑜的愛女韓冰昨(28日)受邀上政論節目《54新觀點》，首度與民進黨台北市議員王...,不用跳愛河！韓冰邀替高雄拚觀光 王世堅靦腆：可以廢物利用
4,政治,點我直接觀看 被操弄的民調？《蘋果》數據解密,【民調解密3】林佳龍敗在紫爆 盧秀燕跌破眼鏡大勝


## 分詞/分句子，把文章轉換成Vector
- 清理文章，再分成句子
- 把句子轉換成count vector

In [3]:
# 選擇文章，把文章分成句子
docNum = 31
testDoc = news100['content'][docNum]
testTitle = news100['title'][docNum]

import re
def splitSentences(document):
    #for sent in re.findall(u'[^!?。\.\!\?]+[!?。\.\!\?]?', paragraph, flags=re.U):
    for sent in re.findall(u'[^!?。\!\?\？]+[!?。\!\?\？]?', document, flags=re.U):
        yield sent

import jieba
sentenceDoc = []
for sent in list(splitSentences(testDoc))[:-1]:
    sentenceDoc.append(' '.join(jieba.cut(sent)))

# 原本文章
testDoc

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\YB0002~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.744 seconds.
Prefix dict has been built succesfully.


'今年九合一大選投票狀況多，據了解，中選會官方網站在投票日當天曾遭到來自境外大規模分散式阻斷服務（DD0oS）攻擊，所幸在負責網路維護中華電信應變下，未進一步造成影響。中選會副主委陳朝建今證實曾遭駭客攻擊，不過計票網路是封閉系統，並非公開網路，無法由外界接取，因此沒有任何影響，境外網路干擾也並未影響投票進程。行政院發言人Kolas Yotaka（谷辣斯・尤達卡）表示，中選會在選前即做好縝密防護，因此成功防堵攻擊，接下來將針對有形的選務如投票排隊問題，以及無形的選務如資安防護進行檢討。不過，投票日當天開票進程緩慢，台北市選情更創下史上第一次開票長達9小時的紀錄，陳朝建說明，開票時間長是因為投票時間延後，計票不受駭客影響，開票也自然不受此影響。據內部消息指出，投票日當天曾出現從境外進行分散式阻斷服務（DDoS）攻擊，企圖癱瘓中選會網站，不過在中華電信、中選會緊急處理後，阻斷來自國外異常流量，在短時間獲得解決，一般民眾並沒有發現特殊異樣。NCC及中華電信暫未對此做進一步說明。陳朝建表示，受攻擊的是中選會官方網站，當日開票的計票網路是獨立的封閉網路，並非公開網路，無法由外界接取，完全未有任何影響；中華電信團隊也是24小時監控與防護作業，也沒有受到任何影響。陳朝建說，受到攻擊的官網，當日皆有隨時防衛，中華電信團隊24小時監控與防護作業；故當日遭到來自境外大規模分散式阻斷服務（DD0oS）攻擊時，在負責網路維護中華電信的緊急應變下，並沒有受到影響。另外，Kolas表示，中選會的官網在選舉當天上午6時56分至7時01分期間，遭受大規模境外DDoS攻擊，而中選會的系統是封閉系統，防護非常嚴密，因此沒被攻擊，而當因中選會在選前即做好縝密的防護，當天其實沒因網路攻擊發生意外狀況，中選會的資安管理有成功防堵攻擊。Kolas表示，所幸這次在事前有預見可能的危險，中選會的資料庫、網站等都沒遭受損害，而在資訊時代，行政院希望在這波中選會的選務檢討中，不只針對有形被詬病的排隊投票問題，無形的選務如資安防護，也會進行檢討、加強。（林惟崧、鄭鴻達／台北報導）出版：1454更新：1942（新增行政院說法） \xa0想知道更多，一定要看……【獨家】投票日遭境外駭客攻擊\u3000中選會網站險癱瘓看了這則新聞的人，也看了……老里長險勝1票\u3000美女里長候選人要求驗票發祭品囉！\u3000韓總雞排逾千

In [4]:
# 分詞後的文章
sentenceDoc

['今年 九 合一 大選 投票 狀況 多 ， 據 了解 ， 中選會 官方 網站 在 投票 日當天 曾 遭到 來 自 境外 大規模 分散式 阻斷 服務 （ DD0oS ） 攻擊 ， 所幸 在 負責 網路 維護 中華電信 應變 下 ， 未進 一步 造成 影響 。',
 '中選會 副 主委 陳 朝 建今 證實 曾 遭駭客 攻擊 ， 不過 計票 網路 是 封閉 系統 ， 並 非公 開網 路 ， 無法 由 外界 接取 ， 因此 沒有 任何 影響 ， 境外 網路 干擾 也 並未 影響 投票 進程 。',
 '行政院 發言人 Kolas   Yotaka （ 谷辣斯 ・ 尤達卡 ） 表示 ， 中選會 在 選前 即 做好 縝密 防護 ， 因此 成功 防堵 攻擊 ， 接下 來 將針 對 有形 的 選務如 投票 排隊 問題 ， 以及 無形 的 選務 如資 安防 護進行 檢討 。',
 '不過 ， 投票 日當天 開票 進程 緩慢 ， 台北市 選情 更 創下史 上 第一次 開票 長 達 9 小時 的 紀錄 ， 陳 朝建 說明 ， 開票 時間 長 是 因為 投票 時間 延 後 ， 計票 不受 駭客 影響 ， 開票 也 自然 不受 此 影響 。',
 '據 內部 消息 指出 ， 投票 日當天 曾 出現 從 境外 進行 分散式 阻斷 服務 （ DDoS ） 攻擊 ， 企圖 癱瘓 中選會 網站 ， 不過 在 中華電信 、 中選會 緊急 處理 後 ， 阻斷 來 自國外 異常 流量 ， 在 短 時間 獲得 解決 ， 一般 民眾 並沒有 發現 特殊 異樣 。',
 'NCC 及 中華電信 暫未 對 此 做 進 一步 說明 。',
 '陳 朝建 表示 ， 受 攻擊 的 是 中 選會 官方 網站 ， 當日 開票 的 計票 網路 是 獨立 的 封閉 網路 ， 並 非公 開網 路 ， 無法 由 外界 接取 ， 完全 未有 任何 影響 ； 中華電信 團隊 也 是 24 小時 監控 與 防護 作業 ， 也 沒 有 受到 任何 影響 。',
 '陳 朝建 說 ， 受到 攻擊 的 官網 ， 當日 皆 有 隨時 防衛 ， 中華電信 團隊 24 小時 監控 與 防護 作業 ； 故當 日 遭到 來 自 境外 大規模 分散式 阻斷 服務 （ DD0oS ） 攻擊時 ， 在 負責 網路 維護 中華電信 的 緊急 應變 下 ， 並

In [24]:
# 把文章轉換成 count vector
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
doc_vector = vect.fit_transform(sentenceDoc)

In [6]:
# 計算 vector 之間的相似度(cosine similarity)
from sklearn.metrics.pairwise import cosine_similarity
cs = cosine_similarity(doc_vector)
cs

array([[1.        , 0.29032258, 0.13116517, 0.18146655, 0.37387825,
        0.16064387, 0.23308662, 0.44901326, 0.18247917, 0.17255919],
       [0.29032258, 1.        , 0.13116517, 0.23331413, 0.17255919,
        0.        , 0.52444489, 0.1796053 , 0.27371875, 0.0862796 ],
       [0.13116517, 0.13116517, 1.        , 0.05270463, 0.11694107,
        0.        , 0.08885233, 0.06085806, 0.34780417, 0.46776428],
       [0.18146655, 0.23331413, 0.05270463, 1.        , 0.13867505,
        0.06454972, 0.2575611 , 0.09622504, 0.        , 0.04622502],
       [0.37387825, 0.17255919, 0.11694107, 0.13867505, 1.        ,
        0.07161149, 0.07792865, 0.26688026, 0.18302666, 0.17948718],
       [0.16064387, 0.        , 0.        , 0.06454972, 0.07161149,
        1.        , 0.07254763, 0.1490712 , 0.        , 0.        ],
       [0.23308662, 0.52444489, 0.08885233, 0.2575611 , 0.07792865,
        0.07254763, 1.        , 0.45962736, 0.22662355, 0.05195243],
       [0.44901326, 0.1796053 , 0.0608580

In [7]:
# 選擇摘要的長度
sentCount = 3

# 用 k-means clustering 把每個 vector 分成三群
from sklearn.cluster import KMeans
km = KMeans(n_clusters=sentCount, init='k-means++')
c = km.fit_predict(cs)
c

array([2, 0, 1, 0, 2, 2, 0, 2, 1, 1])

In [8]:
import numpy as np
np_doc = np.array(sentenceDoc)
print(np_doc[c==1])

['行政院 發言人 Kolas   Yotaka （ 谷辣斯 ・ 尤達卡 ） 表示 ， 中選會 在 選前 即 做好 縝密 防護 ， 因此 成功 防堵 攻擊 ， 接下 來 將針 對 有形 的 選務如 投票 排隊 問題 ， 以及 無形 的 選務 如資 安防 護進行 檢討 。'
 '另外 ， Kolas 表示 ， 中選會 的 官網 在 選舉 當天 上午 6 時 56 分至 7 時 01 分期 間 ， 遭受 大規模 境外 DDoS 攻擊 ， 而 中 選會 的 系統 是 封閉 系統 ， 防護 非常 嚴密 ， 因此 沒 被 攻擊 ， 而當 因中 選會 在 選前 即 做好 縝密 的 防護 ， 當天 其實 沒因 網路 攻擊發 生意 外狀況 ， 中選會 的 資安 管理 有 成功 防堵 攻擊 。'
 'Kolas 表示 ， 所幸 這次 在 事前 有 預見 可能 的 危險 ， 中選會 的 資料 庫 、 網站 等 都 沒 遭受 損害 ， 而 在 資訊 時代 ， 行政院 希望 在 這波 中選會 的 選務 檢討 中 ， 不 只針 對 有形 被 詬 病 的 排隊 投票 問題 ， 無形 的 選務 如資 安防 護 ， 也 會 進行 檢討 、 加強 。']


## 分群
- 把每個句子分成群依照句子跟中心的相似度

In [9]:
# 把每個分群依照跟中心的相似度排列從大到小，找出最適合做摘要的句子
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
order_centroids

array([[6, 1, 3, 7, 0, 8, 4, 2, 9, 5],
       [2, 9, 8, 1, 0, 4, 6, 7, 3, 5],
       [0, 7, 4, 5, 6, 1, 8, 3, 9, 2]], dtype=int64)

In [10]:
# 找出每個群的前三個句子(i.e. 最重要/相似的句子)
centroid_top3 = km.cluster_centers_.argsort()[:, :6:-1]
centroid_top3

array([[6, 1, 3],
       [2, 9, 8],
       [0, 7, 4]], dtype=int64)

In [11]:
# 依照每個群print句子
print('標題: ' + '《' + testTitle + '》')
clusterIndex = 0
for cluster in centroid_top3:
    print('Cluster: ' + str(clusterIndex))
    for sent in cluster:
        print('Sentence ' + str(sent), sentenceDoc[sent].replace(' ','').strip())
    clusterIndex += 1
    print('')

標題: 《中選會證實投票日遭駭客攻擊　行政院：成功防堵》
Cluster: 0
Sentence 6 陳朝建表示，受攻擊的是中選會官方網站，當日開票的計票網路是獨立的封閉網路，並非公開網路，無法由外界接取，完全未有任何影響；中華電信團隊也是24小時監控與防護作業，也沒有受到任何影響。
Sentence 1 中選會副主委陳朝建今證實曾遭駭客攻擊，不過計票網路是封閉系統，並非公開網路，無法由外界接取，因此沒有任何影響，境外網路干擾也並未影響投票進程。
Sentence 3 不過，投票日當天開票進程緩慢，台北市選情更創下史上第一次開票長達9小時的紀錄，陳朝建說明，開票時間長是因為投票時間延後，計票不受駭客影響，開票也自然不受此影響。

Cluster: 1
Sentence 2 行政院發言人KolasYotaka（谷辣斯・尤達卡）表示，中選會在選前即做好縝密防護，因此成功防堵攻擊，接下來將針對有形的選務如投票排隊問題，以及無形的選務如資安防護進行檢討。
Sentence 9 Kolas表示，所幸這次在事前有預見可能的危險，中選會的資料庫、網站等都沒遭受損害，而在資訊時代，行政院希望在這波中選會的選務檢討中，不只針對有形被詬病的排隊投票問題，無形的選務如資安防護，也會進行檢討、加強。
Sentence 8 另外，Kolas表示，中選會的官網在選舉當天上午6時56分至7時01分期間，遭受大規模境外DDoS攻擊，而中選會的系統是封閉系統，防護非常嚴密，因此沒被攻擊，而當因中選會在選前即做好縝密的防護，當天其實沒因網路攻擊發生意外狀況，中選會的資安管理有成功防堵攻擊。

Cluster: 2
Sentence 0 今年九合一大選投票狀況多，據了解，中選會官方網站在投票日當天曾遭到來自境外大規模分散式阻斷服務（DD0oS）攻擊，所幸在負責網路維護中華電信應變下，未進一步造成影響。
Sentence 7 陳朝建說，受到攻擊的官網，當日皆有隨時防衛，中華電信團隊24小時監控與防護作業；故當日遭到來自境外大規模分散式阻斷服務（DD0oS）攻擊時，在負責網路維護中華電信的緊急應變下，並沒有受到影響。
Sentence 4 據內部消息指出，投票日當天曾出現從境外進行分散式阻斷服務（DDoS）攻擊，企圖癱瘓中選會網站，不過在中華電信、中選會緊急處理後，阻斷來自國外異常流量，在短時間獲得解決，一

## 長文摘要

In [12]:
# 找出每個cluster最像中心的三個句子做摘要
summarySentences = []
for cluster in order_centroids:
    summarySentences.append(cluster[0])

print('標題: ' + '《' + testTitle + '》')
for sentence in sorted(summarySentences):
    print('Sentence ' + str(sentence) + '\n', sentenceDoc[sentence].replace(' ','').strip())

標題: 《中選會證實投票日遭駭客攻擊　行政院：成功防堵》
Sentence 0
 今年九合一大選投票狀況多，據了解，中選會官方網站在投票日當天曾遭到來自境外大規模分散式阻斷服務（DD0oS）攻擊，所幸在負責網路維護中華電信應變下，未進一步造成影響。
Sentence 2
 行政院發言人KolasYotaka（谷辣斯・尤達卡）表示，中選會在選前即做好縝密防護，因此成功防堵攻擊，接下來將針對有形的選務如投票排隊問題，以及無形的選務如資安防護進行檢討。
Sentence 6
 陳朝建表示，受攻擊的是中選會官方網站，當日開票的計票網路是獨立的封閉網路，並非公開網路，無法由外界接取，完全未有任何影響；中華電信團隊也是24小時監控與防護作業，也沒有受到任何影響。


**Yahoo新聞文章**

In [13]:
rand_title =  '柯P選總統 打趴藍綠無敵手'
rand_doc   = ('綠營背景的新台灣國策智庫，昨發布「九合一選後最新政局情形與2020總統大選選情評估」民調，支持台北市長柯文哲參選2020大選者，48.2％，「KO」所有政治人物包括國民黨主席吳敦義、蔡總統、新北市長朱立倫、行政院長賴清德，及準高雄市長韓國瑜。'
              '應邀分析民調的成大政治系副教授蒙志成指出，以柯為典型的第三勢力已有與藍綠支持者抗衡基礎，如明天就是總統大選，不管對手是誰，柯均有機會獲勝。'
              '蒙志成說，值得觀察的是，當藍綠都派出最強候選人呈現三強鼎立，藍綠候選人仍可一搏，絕非無勝選機會；但如出現一強一弱，選民會產生棄保策略，而使柯文哲成最大受益者。'
              '民調顯示，如柯對上民進黨蔡英文與國民黨吳敦義，柯獲53.3％支持，蔡19.1％，吳14.1％，若柯對上朱立倫和賴清德，柯是34.3％支持，朱29.5％，賴28.2％。若國民黨改推「韓流」，柯、賴及韓支持度3人不相上下，分別為31.8％、30.5％、29.2％，可見「韓粉」與「柯粉」高度重疊，柯僅能「險勝」；而30至39歲民眾支持柯更高達74.2％。政黨傾向分析，最支持柯選總統的是親民黨支持者，62.5％；排名第2是「自認不偏任何黨」，49.9％支持。'
              '反之，各政黨中最不樂見柯選總統的多為民進黨支持者，55.4％不支持，但仍有40.5％支持，顯見柯對泛綠仍有吸引力。')

# Yahoo news
# https://tw.news.yahoo.com/%E4%BE%AF%E5%8F%8B%E5%AE%9C-%E4%B8%8D%E6%8E%92%E9%99%A4%E8%88%87%E6%9F%AF%E6%96%87%E5%93%B2-%E8%94%A1%E7%B8%BD%E7%B5%B1%E5%90%88%E4%BD%9C-042011077.html
title1 =  '侯友宜：不排除與柯文哲、蔡總統合作'
doc1   = ('準新北市長侯友宜小內閣團隊布局就位，今（21日）10點公佈小內閣名單，備受矚目的三名副市長人選，謝政達、吳明機及內政部前政次陳純敬接任，在24位局處首長中，有5位是從內部升上來。對於未來雙北是否合作及何時與總統蔡英文會面，侯友宜說，只為雙北市民好，隨時都可以去見柯市長，至於跟在總統會面，他表示25號就職後會更積極與府方聯絡。'
              '發布小內閣後，媒體提問跟台北市柯文哲會不會雙北合作？侯友宜表示，他隨時可以去看柯文哲，「我車搭了就可以去看他，做朋友有太多不需要拘泥太多的形式和規則」，為了雙北市民好，我隨時都可以去見柯市長，為雙北做事沒有眉角，好好的做事，「如果有天，看我開車去看柯市長沒通知你不要怪我」，因為都好朋友，走來走去，都是好朋友。'
              '至於何時跟蔡總統會面？侯友宜說，「中央地方本來就要綁作伙、鬥陣行，好好的努力拚我們的建設，拚我們的經濟，這才是我要的」，他強調也會跟中央合作，跟柯市長合作，跟任何各縣市合作、鬥陣來，「台灣小小的，中華民國小小的，每個都是厝邊隔壁，都是親戚，走出去都是我們的朋友」，共同為這塊土地來貢獻。'
              '侯友宜表示，和蔡總統會面的時間和地點都沒敲好，因為和府方表示25日就職後，找一個地方，時間也要配合總統時間，看哪個建設，如何為新北市民來做。所以上任後，會更積極與府方聯絡。')

# Setn news
# https://www.setn.com/News.aspx?NewsID=475058
title2 = '柯文哲若尋求下任總統　民調：4成4民眾支持、5成反對'
doc2   = ('九合一選舉過後，誰來角逐2020引發關注。台灣民意調查基金會今（24）日公布民調顯示，在政治人物的好感度方面，剛連任的台北市長柯文哲以（65.81度）領先高雄市長韓國瑜等政治人物。至於如果柯P尋求參選下任總統，民調顯示，有4成4的民眾支持，但有5成的人反對，反對的人多於支持的人。'
          '台灣民意基金會董事長游盈隆今天上午召開「2018年終台灣重大民意走向」記者會。此次調查重點包括蔡英文總統聲望、賴清德內閣的施政表現、蔡英文總統兩岸表現的民意反應、台灣人對主要政治人的感情溫度、台灣人對2020主要總統可能人選的支持傾向、台灣人的統獨傾向、台灣人的政黨認同。'
          '關於台灣人對主要政治人物的感情溫度，民調顯示，台北市長最高（65.81度）、其次是高雄市長韓國瑜（62.12度），第三是新北市長侯友宜（59.79度）、第四是行政院長賴清德（53.818度）、第五是蔡英文總統（42.98度）。'
          '至於柯文哲若參選下任總統？民調顯示，有18.3%非常支持，25.5%還算支持，25.1%不太支持，24.8%一點也不支持， 6.3%沒意見、不知道、拒答。換句話說，在二十歲以上的台灣人當中，有四成四的人基 本上支持柯文哲參選下一屆總統，但有五成的人反對，反對的人多於支持的人。'
          '該民調由山水民意研究公進行電話訪談與統計分析，訪問期間是17-18 兩天，以全國為範圍的二十歲以上成年人，有效樣本 1082 人；抽樣誤差在 95%信心水準下約正負 2.98 個百分點。')

# Yahoo news
# https://tw.news.yahoo.com/%E9%9B%99%E5%9F%8E%E8%AB%96%E5%A3%87%E6%99%9A%E5%AE%B4-%E6%9F%AF%E6%96%87%E5%93%B2%E9%87%8D%E6%8F%90%E5%85%A9%E5%B2%B8%E4%B8%80%E5%AE%B6%E8%A6%AA-113314791.html
title3 =  '雙城論壇晚宴 柯文哲重提兩岸一家親'
doc3   = ('台北上海雙城論壇歡迎晚宴今天晚間在圓山大飯店舉行，台北市長柯文哲致詞時重提五個互相，並表示雙方應秉持「兩岸一家親」精神交流，兩岸關係就可和平穩定。上海市常務副市長周波也表示，上海與台北拓展交流合作，一定會有更多成果，兩市人民也會在「兩岸一家親」氛圍中，共同分享「珍貴親情和友情」。'
          '上海市常務副市長周波上午率團抵台，參訪台達電後，晚間赴圓山大飯店參加歡迎晚宴，分別與台北市長柯文哲在晚宴上致詞，並互贈禮物。柯文哲致詞時表示，上海方此次派出130人與會，台北市產官學代表有250人，參與人數超越歷屆，盛況空前，代表大家對雙城論壇的重視，也因雙方有互信基礎，才能在這麼短時間內舉行。'
          '他說，台北市與上海市過去舉辦8次雙城論壇，透過交流來互相認識、互相了解，不但拉近距離，展開良好互動，更經由互相尊重、互相合作，簽署30項備忘錄，深化各領域交流，成果豐碩。「過去4年，兩岸雖然政治形勢不斷變化，但是雙城論壇還是持續舉辦，關鍵就在互相諒解，體諒彼此面對的困難」，柯文哲說，在善意持續交流，在交流中累積信任，「雙城論壇才能在一次又一次挑戰下，走到今天」。'
          '他表示，盼未來雙方在既有政治基礎上，維繫得來不易的交流成果，持續發揮智慧和創意，讓雙城論壇長長久久，永續發展，成為兩岸關係和平發展的典範，柯文哲強調，只要秉持五個互相以及兩岸一家親精神，兩岸關係就會和平穩定，也可達互惠互利局面。'
          '柯文哲說，兩岸人民同文同種，擁有共同歷史文化，「兩岸關係好，符合兩岸人民的福祉，也有利於台灣的經濟發展」，認為只要有更多善意交流，很多問題可以解決，或者避免。他說，兩岸交流已超過30年，兩岸間的經貿文化交流已密不可分，中國大陸已是世界第2大經濟體，上海表現更是亮眼，向世界展現開放與創新實力，台北過去30年發展，也展現開放、包容的生命力，確立獨特城市魅力。'
          '柯文哲說：「台北與上海各有千秋，台北的多元與包容、上海的繁榮與創新，希望雙方交流可達到共好共融目標」。柯文哲也宣示，台北上海雙城論壇是兩岸交流典範，未來仍將常態性辦理，「為兩岸和平發展作出貢獻」。'
          '上海市常務副市長周波表示，相信隨著兩市的進一步交流合作，一定會有更多的成果展現，「兩市的人民也在這種『兩岸一家親』的氛圍當中，共同分享這珍貴的親情和友情」。'
          '他表示，明天雙城論壇將簽署新的交流合作備忘錄，相信在雙方共同努力和精心的呵護下，上海和台北的交流合作一定會更加的深入，更加的廣泛，更富成效，兩市人民的友誼和感情，一定會越來越醇厚。')

# Yahoo news
# https://tw.news.yahoo.com/%E6%9F%AF%E6%96%87%E5%93%B2-%E5%AE%9A%E6%9C%83%E8%A6%8B%E9%9F%93%E5%9C%8B%E7%91%9C-%E4%B9%9F%E6%87%89%E8%A9%B2%E6%8B%9C%E8%A8%AA%E4%BE%AF%E5%8F%8B%E5%AE%9C-071621335.html
title4 = '柯文哲：一定會見韓國瑜 也應該拜訪侯友宜'
doc4   = ('（中央社記者陳妍君台北14日電）媒體報導，台北市長柯文哲預定明年元月與新任高雄市長韓國瑜會面。柯文哲今天說，幾個大城市首長還是要見面，他一定會見韓國瑜，也應該找時間拜訪新北市長當選人侯友宜。對於1月是否將與韓國瑜會面，柯文哲上午受訪時表示：「一定會的，只是我們大家都很忙」；至於他是否將前往高雄，柯文哲未正面回應，僅說韓國瑜「也常常來台北啊」。'
          '柯文哲下午出席台北市公私立中等學校優良學生表揚大會前又受訪。他表示，等大家忙完，幾個大城市首長還是要見面，「老實講我也應該找個時間去拜訪侯友宜」。另外，媒體問到，市府觀光傳播局長陳思宇上午登記參加台北市立委補選，宣告白綠合作破局，柯文哲說：「其實我們早就準備好了啦，我們一直在等嘛。可是一直等到昨天晚上也沒有人來跟我們討論啊，所以今天就去登記了。」'
          '媒體追問，他是否指民進黨沒有人找他討論白綠合作事宜。柯文哲表示，不曉得到底現在民進黨是誰在做決定，「各種聲音都有，只是好像也沒有人可以來跟我們商量。所以沒辦法，她去登記了」。媒體詢問，陳思宇登記前受訪表示，會跟柯文哲說，「我在中央等著你」，被認為是暗示他參選2020年總統，他怎麼看。'
          '柯文哲笑稱，「要跟她講說不要亂講話」；「我覺得那個無聊啊，好啦好啦」。對於2020年要不要支持總統蔡英文，柯文哲說，這種事情哪裡需要現在回答，到時候再講就好了；政壇上的東西都是隨便說說，大家守信度都很差，「我都不太想做這種事」。')

# Setn news
# https://www.setn.com/News.aspx?NewsID=472836
title5 =  '九合一嗆殺韓國瑜、柯文哲走路工　警政署：列重大案件'
doc5   = ('九合一選舉期間，警方積極查賄及嚴查假訊息，根據內政部統計，截至12月17日止，各警察機關已移送賄選案件1,275件6,983人、網路不實言論案件138件174人，其中選前在PTT論壇上嗆殺準高雄市長韓國瑜的趙姓男子，與在臉書社團說要替台北市長柯文哲動員走路工發500元的張姓、曹姓兩名魔術師都遭送辦。'
          '內政部表示，這次查賄工作由警察機關主動出擊，選前各警察機關即主動布雷，現仍持續配合地檢署辦理查賄移送作業，及幽靈人口清查偵辦工作。截至12月17日止，各警察機關已移送賄選案件1,275件6,983人，202人經檢方向法院聲請羈押獲准，其中候選人身分為69人。內政部指出，這次9合1選舉，候選人多運用網路作選舉政見論述，部分也遭有心人士以虛偽不實言論散布假消息，企圖影響選舉，可能觸犯公職人員選舉罷免法、刑法或社會秩序維護法。'
          '為此，刑事警察局特別成立「查緝假消息專責小組」，並要求各縣市警察局加強偵辦網路不實言論假消息。截至12月17日止，警方已受理網路不實言論案件166件，已偵破138件174人移送各相關地檢署，並有28件持續蒐證調查中。其中，由「查緝假消息專責小組」偵辦涉選舉之重大網路不實言論案件共11件，已偵破6件移送各相關地檢署，並有5件持續偵辦中。'
          '警方偵辦的相關重大案例包括，選前刑事警察局發現有不明人士於PTT論壇上針對本屆高雄市長候選人發表恐嚇言論，內容已涉及刑法恐嚇危害安全罪嫌，警方迅速鎖定嫌疑人，到案後依法移送士林地方檢察署偵辦。另外，台北市柯文哲選前疑遭不明人士於臉書及LINE群組發送徵招工作人員及走路工訊息，經競選辦公室工作人員提告後，由台北市政府警察局迅速鎖定兩嫌，到案後依公職人員選舉罷免法第104條移送台北地方檢察署偵辦。'
          '另外，內政部並表示，也有不明人士於PTT論壇上，質疑三鳳宮假籤詩是基進黨候選人造假，經基進黨候選人至高雄地方檢察署按鈴申告後，由高雄市政府警察局迅速鎖定嫌疑人，到案後依公職人員選舉罷免法第104條移送高雄地方檢察署偵辦。警政署表示，對於在各類型社群網路平台發表影響選舉的不實言論案件，一經當事人提告，警方將全力依法偵辦，呼籲民眾於社群網路、通訊軟體發言時應謹慎，避免引起社會恐慌或造成他人名譽受損。')

In [29]:
# cluster-based 摘要，輸入文章的號碼和摘要長度
def clusterSummarizer(title, document, sumLength):
    testTitle = title
    testDoc = document

    sentenceDoc = []
    for sent in list(splitSentences(testDoc))[:-1]:
        sentenceDoc.append(' '.join(jieba.cut(sent)))
    
    doc_vector = vect.fit_transform(sentenceDoc)
    print(doc_vector[2])
    cs = cosine_similarity(doc_vector)
    
    km = KMeans(n_clusters=sumLength, init='k-means++')
    c = km.fit_predict(cs)
    order_centroids = km.cluster_centers_.argsort()[:, ::-1]
    
    np_doc = np.array(sentenceDoc)
    #print(np_doc[c==0], '\n\n')
    
    summarySentences = []
    for cluster in order_centroids:
        summarySentences.append(cluster[0])
        
    print('標題: ' + '《' + testTitle + '》')
    for sentence in sentenceDoc:
        print(sentenceDoc.index(sentence),sentence.replace(' ','').strip())

    print('')
        
    print('標題: ' + '《' + testTitle + '》')
    for sentence in sorted(summarySentences):
        print('Sentence ' + str(sentence) + '\n', sentenceDoc[sentence].replace(' ','').strip())

In [15]:
clusterSummarizer(title1,doc1,4)
#clusterSummarizer(news100['title'][31],news100['content'][31],3)

標題: 《侯友宜：不排除與柯文哲、蔡總統合作》
0 準新北市長侯友宜小內閣團隊布局就位，今（21日）10點公佈小內閣名單，備受矚目的三名副市長人選，謝政達、吳明機及內政部前政次陳純敬接任，在24位局處首長中，有5位是從內部升上來。
1 對於未來雙北是否合作及何時與總統蔡英文會面，侯友宜說，只為雙北市民好，隨時都可以去見柯市長，至於跟在總統會面，他表示25號就職後會更積極與府方聯絡。
2 發布小內閣後，媒體提問跟台北市柯文哲會不會雙北合作？
3 侯友宜表示，他隨時可以去看柯文哲，「我車搭了就可以去看他，做朋友有太多不需要拘泥太多的形式和規則」，為了雙北市民好，我隨時都可以去見柯市長，為雙北做事沒有眉角，好好的做事，「如果有天，看我開車去看柯市長沒通知你不要怪我」，因為都好朋友，走來走去，都是好朋友。
4 至於何時跟蔡總統會面？
5 侯友宜說，「中央地方本來就要綁作伙、鬥陣行，好好的努力拚我們的建設，拚我們的經濟，這才是我要的」，他強調也會跟中央合作，跟柯市長合作，跟任何各縣市合作、鬥陣來，「台灣小小的，中華民國小小的，每個都是厝邊隔壁，都是親戚，走出去都是我們的朋友」，共同為這塊土地來貢獻。
6 侯友宜表示，和蔡總統會面的時間和地點都沒敲好，因為和府方表示25日就職後，找一個地方，時間也要配合總統時間，看哪個建設，如何為新北市民來做。

標題: 《侯友宜：不排除與柯文哲、蔡總統合作》
Sentence 0
 準新北市長侯友宜小內閣團隊布局就位，今（21日）10點公佈小內閣名單，備受矚目的三名副市長人選，謝政達、吳明機及內政部前政次陳純敬接任，在24位局處首長中，有5位是從內部升上來。
Sentence 1
 對於未來雙北是否合作及何時與總統蔡英文會面，侯友宜說，只為雙北市民好，隨時都可以去見柯市長，至於跟在總統會面，他表示25號就職後會更積極與府方聯絡。
Sentence 2
 發布小內閣後，媒體提問跟台北市柯文哲會不會雙北合作？
Sentence 4
 至於何時跟蔡總統會面？


In [30]:
clusterSummarizer(title2,doc2,4)

  (0, 36)	1
  (0, 57)	2
  (0, 69)	1
  (0, 80)	1
  (0, 17)	1
  (0, 35)	1
  (0, 51)	1
  (0, 50)	1
  (0, 82)	1
  (0, 102)	1
  (0, 70)	1
標題: 《柯文哲若尋求下任總統　民調：4成4民眾支持、5成反對》
0 九合一選舉過後，誰來角逐2020引發關注。
1 台灣民意調查基金會今（24）日公布民調顯示，在政治人物的好感度方面，剛連任的台北市長柯文哲以（65.81度）領先高雄市長韓國瑜等政治人物。
2 至於如果柯P尋求參選下任總統，民調顯示，有4成4的民眾支持，但有5成的人反對，反對的人多於支持的人。
3 台灣民意基金會董事長游盈隆今天上午召開「2018年終台灣重大民意走向」記者會。
4 此次調查重點包括蔡英文總統聲望、賴清德內閣的施政表現、蔡英文總統兩岸表現的民意反應、台灣人對主要政治人的感情溫度、台灣人對2020主要總統可能人選的支持傾向、台灣人的統獨傾向、台灣人的政黨認同。
5 關於台灣人對主要政治人物的感情溫度，民調顯示，台北市長最高（65.81度）、其次是高雄市長韓國瑜（62.12度），第三是新北市長侯友宜（59.79度）、第四是行政院長賴清德（53.818度）、第五是蔡英文總統（42.98度）。
6 至於柯文哲若參選下任總統？
7 民調顯示，有18.3%非常支持，25.5%還算支持，25.1%不太支持，24.8%一點也不支持，6.3%沒意見、不知道、拒答。
8 換句話說，在二十歲以上的台灣人當中，有四成四的人基本上支持柯文哲參選下一屆總統，但有五成的人反對，反對的人多於支持的人。

標題: 《柯文哲若尋求下任總統　民調：4成4民眾支持、5成反對》
Sentence 0
 九合一選舉過後，誰來角逐2020引發關注。
Sentence 2
 至於如果柯P尋求參選下任總統，民調顯示，有4成4的民眾支持，但有5成的人反對，反對的人多於支持的人。
Sentence 3
 台灣民意基金會董事長游盈隆今天上午召開「2018年終台灣重大民意走向」記者會。
Sentence 5
 關於台灣人對主要政治人物的感情溫度，民調顯示，台北市長最高（65.81度）、其次是高雄市長韓國瑜（62.12度），第三是新北市長侯友宜（59.79度）、第四是行政院長賴清德（

In [17]:
clusterSummarizer(title3,doc3,4)

標題: 《雙城論壇晚宴 柯文哲重提兩岸一家親》
0 台北上海雙城論壇歡迎晚宴今天晚間在圓山大飯店舉行，台北市長柯文哲致詞時重提五個互相，並表示雙方應秉持「兩岸一家親」精神交流，兩岸關係就可和平穩定。
1 上海市常務副市長周波也表示，上海與台北拓展交流合作，一定會有更多成果，兩市人民也會在「兩岸一家親」氛圍中，共同分享「珍貴親情和友情」。
2 上海市常務副市長周波上午率團抵台，參訪台達電後，晚間赴圓山大飯店參加歡迎晚宴，分別與台北市長柯文哲在晚宴上致詞，並互贈禮物。
3 柯文哲致詞時表示，上海方此次派出130人與會，台北市產官學代表有250人，參與人數超越歷屆，盛況空前，代表大家對雙城論壇的重視，也因雙方有互信基礎，才能在這麼短時間內舉行。
4 他說，台北市與上海市過去舉辦8次雙城論壇，透過交流來互相認識、互相了解，不但拉近距離，展開良好互動，更經由互相尊重、互相合作，簽署30項備忘錄，深化各領域交流，成果豐碩。
5 「過去4年，兩岸雖然政治形勢不斷變化，但是雙城論壇還是持續舉辦，關鍵就在互相諒解，體諒彼此面對的困難」，柯文哲說，在善意持續交流，在交流中累積信任，「雙城論壇才能在一次又一次挑戰下，走到今天」。
6 他表示，盼未來雙方在既有政治基礎上，維繫得來不易的交流成果，持續發揮智慧和創意，讓雙城論壇長長久久，永續發展，成為兩岸關係和平發展的典範，柯文哲強調，只要秉持五個互相以及兩岸一家親精神，兩岸關係就會和平穩定，也可達互惠互利局面。
7 柯文哲說，兩岸人民同文同種，擁有共同歷史文化，「兩岸關係好，符合兩岸人民的福祉，也有利於台灣的經濟發展」，認為只要有更多善意交流，很多問題可以解決，或者避免。
8 他說，兩岸交流已超過30年，兩岸間的經貿文化交流已密不可分，中國大陸已是世界第2大經濟體，上海表現更是亮眼，向世界展現開放與創新實力，台北過去30年發展，也展現開放、包容的生命力，確立獨特城市魅力。
9 柯文哲說：「台北與上海各有千秋，台北的多元與包容、上海的繁榮與創新，希望雙方交流可達到共好共融目標」。
10 柯文哲也宣示，台北上海雙城論壇是兩岸交流典範，未來仍將常態性辦理，「為兩岸和平發展作出貢獻」。
11 上海市常務副市長周波表示，相信隨著兩市的進一步交流合作，一定會有更多的成果展現，「兩市的人民也在這種『兩岸一家親』的氛圍當中，共同分享這珍貴的親情和友情」。


In [18]:
clusterSummarizer(title4,doc4,4)

標題: 《柯文哲：一定會見韓國瑜 也應該拜訪侯友宜》
0 （中央社記者陳妍君台北14日電）媒體報導，台北市長柯文哲預定明年元月與新任高雄市長韓國瑜會面。
1 柯文哲今天說，幾個大城市首長還是要見面，他一定會見韓國瑜，也應該找時間拜訪新北市長當選人侯友宜。
2 對於1月是否將與韓國瑜會面，柯文哲上午受訪時表示：「一定會的，只是我們大家都很忙」；至於他是否將前往高雄，柯文哲未正面回應，僅說韓國瑜「也常常來台北啊」。
3 柯文哲下午出席台北市公私立中等學校優良學生表揚大會前又受訪。
4 他表示，等大家忙完，幾個大城市首長還是要見面，「老實講我也應該找個時間去拜訪侯友宜」。
5 另外，媒體問到，市府觀光傳播局長陳思宇上午登記參加台北市立委補選，宣告白綠合作破局，柯文哲說：「其實我們早就準備好了啦，我們一直在等嘛。
6 可是一直等到昨天晚上也沒有人來跟我們討論啊，所以今天就去登記了。
7 」媒體追問，他是否指民進黨沒有人找他討論白綠合作事宜。
8 柯文哲表示，不曉得到底現在民進黨是誰在做決定，「各種聲音都有，只是好像也沒有人可以來跟我們商量。
9 所以沒辦法，她去登記了」。
10 媒體詢問，陳思宇登記前受訪表示，會跟柯文哲說，「我在中央等著你」，被認為是暗示他參選2020年總統，他怎麼看。
11 柯文哲笑稱，「要跟她講說不要亂講話」；「我覺得那個無聊啊，好啦好啦」。

標題: 《柯文哲：一定會見韓國瑜 也應該拜訪侯友宜》
Sentence 1
 柯文哲今天說，幾個大城市首長還是要見面，他一定會見韓國瑜，也應該找時間拜訪新北市長當選人侯友宜。
Sentence 2
 對於1月是否將與韓國瑜會面，柯文哲上午受訪時表示：「一定會的，只是我們大家都很忙」；至於他是否將前往高雄，柯文哲未正面回應，僅說韓國瑜「也常常來台北啊」。
Sentence 5
 另外，媒體問到，市府觀光傳播局長陳思宇上午登記參加台北市立委補選，宣告白綠合作破局，柯文哲說：「其實我們早就準備好了啦，我們一直在等嘛。
Sentence 9
 所以沒辦法，她去登記了」。


In [19]:
clusterSummarizer(title5,doc5,4)

標題: 《九合一嗆殺韓國瑜、柯文哲走路工　警政署：列重大案件》
0 九合一選舉期間，警方積極查賄及嚴查假訊息，根據內政部統計，截至12月17日止，各警察機關已移送賄選案件1,275件6,983人、網路不實言論案件138件174人，其中選前在PTT論壇上嗆殺準高雄市長韓國瑜的趙姓男子，與在臉書社團說要替台北市長柯文哲動員走路工發500元的張姓、曹姓兩名魔術師都遭送辦。
1 內政部表示，這次查賄工作由警察機關主動出擊，選前各警察機關即主動布雷，現仍持續配合地檢署辦理查賄移送作業，及幽靈人口清查偵辦工作。
2 截至12月17日止，各警察機關已移送賄選案件1,275件6,983人，202人經檢方向法院聲請羈押獲准，其中候選人身分為69人。
3 內政部指出，這次9合1選舉，候選人多運用網路作選舉政見論述，部分也遭有心人士以虛偽不實言論散布假消息，企圖影響選舉，可能觸犯公職人員選舉罷免法、刑法或社會秩序維護法。
4 為此，刑事警察局特別成立「查緝假消息專責小組」，並要求各縣市警察局加強偵辦網路不實言論假消息。
5 截至12月17日止，警方已受理網路不實言論案件166件，已偵破138件174人移送各相關地檢署，並有28件持續蒐證調查中。
6 其中，由「查緝假消息專責小組」偵辦涉選舉之重大網路不實言論案件共11件，已偵破6件移送各相關地檢署，並有5件持續偵辦中。
7 警方偵辦的相關重大案例包括，選前刑事警察局發現有不明人士於PTT論壇上針對本屆高雄市長候選人發表恐嚇言論，內容已涉及刑法恐嚇危害安全罪嫌，警方迅速鎖定嫌疑人，到案後依法移送士林地方檢察署偵辦。
8 另外，台北市柯文哲選前疑遭不明人士於臉書及LINE群組發送徵招工作人員及走路工訊息，經競選辦公室工作人員提告後，由台北市政府警察局迅速鎖定兩嫌，到案後依公職人員選舉罷免法第104條移送台北地方檢察署偵辦。
9 另外，內政部並表示，也有不明人士於PTT論壇上，質疑三鳳宮假籤詩是基進黨候選人造假，經基進黨候選人至高雄地方檢察署按鈴申告後，由高雄市政府警察局迅速鎖定嫌疑人，到案後依公職人員選舉罷免法第104條移送高雄地方檢察署偵辦。

標題: 《九合一嗆殺韓國瑜、柯文哲走路工　警政署：列重大案件》
Sentence 0
 九合一選舉期間，警方積極查賄及嚴查假訊息，根據內政部統計，截至12月17日止，各警察機關已移送賄選案件1,27

**會提高摘要效果的改進**
1. 更好的分詞/分句子會有更好的效果和摘要
2. 更進階的方法計算句子的重要性(這個示範只有用詞頻計算相似度)
3. 需要選擇恰當的cluster size(太多clusters摘要會包括多餘的句子，太少clusters會忽略一些重要的句子)
    - 不同的文章會需要不同的cluster size

**其他摘要例子**

In [20]:
# cluster-based 摘要，輸入文章的號碼和摘要長度
def news100_clusterSummarizer(docNumber, sumLength):
    testDoc = news100['content'][docNumber]
    testTitle = news100['title'][docNumber]

    sentenceDoc = []
    for sent in list(splitSentences(testDoc))[:-1]:
        sentenceDoc.append(' '.join(jieba.cut(sent)))
    
    doc_vector = vect.fit_transform(sentenceDoc)
    cs = cosine_similarity(doc_vector)
    
    km = KMeans(n_clusters=sumLength, init='k-means++')
    km.fit_predict(cs)
    order_centroids = km.cluster_centers_.argsort()[:, ::-1]
    
    summarySentences = []
    for cluster in order_centroids:
        summarySentences.append(cluster[0])
        
    print('標題: ' + '《' + testTitle + '》')
    print(testDoc, '\n')

    print('標題: ' + '《' + testTitle + '》')
    for sentence in sorted(summarySentences):
        print('Sentence ' + str(sentence) + '\n', sentenceDoc[sentence].replace(' ','').strip())

In [21]:
# 稍微短一點的文章，第99篇
news100_clusterSummarizer(99,3)

標題: 《柯、侯都愛！副市長虛位以待　邱豐光人氣破表》
選戰落幕，各縣市首長將於12月25日就職，一場搶人大戰正式展開！台北市警察局前局長邱豐光，去年9月被調往警政署任副署長，外界多以「明升暗降」看待。如今選舉結束，台北市長柯文哲、新北市長當選人侯友宜不約而同看上邱，冀望找他來當副市長。《周刊王》報導，柯文哲原本不喜歡邱豐光，後來發現邱「很好用」，不僅對議會的溝通效果比副市長強，邱在商界人脈廣，只要柯P開口，邱使命必達。侯友宜雖是邱豐光警大學弟，但據報導，早在侯決定參選時，已私下探詢邱任副手的意願，當時邱力挺，「只要侯開口，絕對會去。」對此傳聞，柯、侯幕僚均表示目前沒討論。邱豐光回覆「空穴來風」，目前最重要的是做好警政工作。（王嘉慶／綜合報導）想知道更多，一定要看……【君子片】密約曝光！韓國瑜、陳其邁　選前3度交換協議 

標題: 《柯、侯都愛！副市長虛位以待　邱豐光人氣破表》
Sentence 1
 如今選舉結束，台北市長柯文哲、新北市長當選人侯友宜不約而同看上邱，冀望找他來當副市長。
Sentence 3
 侯友宜雖是邱豐光警大學弟，但據報導，早在侯決定參選時，已私下探詢邱任副手的意願，當時邱力挺，「只要侯開口，絕對會去。
Sentence 4
 」對此傳聞，柯、侯幕僚均表示目前沒討論。


In [22]:
# 稍微長一點的文章，第40篇
news100_clusterSummarizer(40,3)

標題: 《坦言「​總統大選光環已過」　蔡英文座談會聚焦基層》
總統蔡英文下午前往宜蘭展開「傾聽之旅」第二站，歷經2小時的座談會，下午5時許蔡英文等人離開會場。被問到是否會參選2020、非核家園2025等問題蔡英文笑而不答，而被問到農委會主委林聰賢是否應該下台一事，她一度欲言又止，最後還是上車離去。 與會的宜蘭縣小英之友會理事曹光炎說，總統放下身段傾聽基層聲音，他相信基層聲音總統都聽到了，民進黨也會虛心改進，尤其是照顧弱勢的農漁工，還說「民進黨的失敗，不是因為國民黨的強，而是因為沒有傾聽人民的聲音」。會中並沒有針對特定人士是否下台進行討論。 民進黨宜蘭縣黨部主委黃進財也說，會中探討執政上遇到、需要改進的問題，並沒有提到行政官員是否下台的部分。他說，各領域都有探討，包括一例一休、農漁會與空污法問題，都有討論到，而執政上沒有立即接地氣與民意站在一起都有做深切檢討。 黃進財也表示，座談會中也多所提及農業政策，包括農地、農舍等議題及農漁民收入、農業產銷穩定等問題都有熱烈討論，總統也相當重視。 另外，宜蘭市長江聰淵表示，在於內閣部分有特別提到，人才尋覓是相當重要課題，過去都被批評是用老宅男、老藍男等問題，人才尋找是當務之急，對於人才培養、部會的養成，因此才會繼續慰留陳菊及賴清德繼續承擔責任。 對於局部調整，也聽很多基層鄉親意見，包括勞工、漁民、政策等問題，中央部會未在第一時間解決，總統承諾未來會經常下鄉聽取民眾聲音，會透過她直接與部會溝通，而非侷限在建議案到她手中後才與部會商量，將來會以下鄉的方式，解決民眾需求，直接了解民眾心聲。 另外，江聰淵也說，許多政策議題上很多有爭議性，總統也處於兩難，包括廢死刑屬國際議題，國際友邦要求不能執行死刑，但國人對於廢死也有反對立場，公投案也處於對立，造成執政者負擔。如今執政運作上問題浮現，須面對問題並解決。 江聰淵說，蔡英文在會中表示「總統大選時的光環已經過去了」，會虛心檢討，鼓勵基層民眾，重拾信心，改革與政策推動，會與地方謀定而後動，最重要的民生議題，包括一例一休等相關政策，會再找出明確性的推動方向。 對於縣黨部輔選不力部分，確實有人提議檢討，江聰淵表示，會中也曾討論到農委會主委，總統說前後兩任均是出身農業大縣的縣長，她給予充分授權，未來要再尋覓有敏感性、能替政府、百姓解決問題的人才，是總統擔憂的部分，但她相信有此時間歷練，各部會都進入狀況

In [23]:
# 效果不太好的摘要，第1篇
news100_clusterSummarizer(1,3)

標題: 《他列民進黨10大豬隊友　「不下架將賠上2020」》
民進黨這次九合一選舉慘敗，蔡英文辭去黨主席一職，昨公布「給黨員的一封信」，內容提到，最該改變的人就是她自己，「責任由我一肩扛起，同志之間不必互相為難。」前立委孫大千則說，「蔡總統又不能辭職下台，那要如何一肩扛起所有責任？這不又是一句空話嗎？」孫大千今天上午在臉書發文，倘若民進黨真的希望能夠救亡圖存，力挽狂瀾，建議在最短的時間內，不論是用勸退、勸辭、或是撤換的方式，讓下列十大「豬隊友」下架，「如果捨不得這十個人，保證會賠上民進黨2020年的政權。」第一，副總統陳建仁理由：823水災時，當南部鄉親還泡在水裡，但是這個人卻在金門玩好玩滿了3天。第二，總統府秘書長陳菊理由：把高雄市整到「又老又窮」，和滿朝盡是「菊家班」。第三，行政院長賴清德理由：「幹話王」的形象早已深植人心。第四，行政院發言人谷辣斯理由：不但有酒駕前科，更不忘記把政府提供給弱勢原住民的法扶資源「用好用完」。第五，駐日代表謝長廷理由：有空回台輔選黨內同志，卻無心援助台灣旅客。第六，交通部長吳宏謀理由：高雄市五千個「天坑」的推手，以及普悠瑪事件的政治責任。第七，農委會副主委李退之理由：豐年社社長汪文豪的「被請辭事件」。第八，監察委員陳師孟理由：替民進黨把司法檢調系統，幾乎一次羞辱、踐踏、得罪完畢。第九，北農總經理吳音寧理由：看到這位有蔡英文當靠山的「高級實習生」，只會讓年輕人一肚子怨氣。第十，中華電信董事長鄭優理由：從「499之亂」，到後續的內部爭議連環爆，他必然將會成為在野黨的下一個靶子。（即時新聞中心／綜合報導）看了這則新聞的人，也看了……14年前撞死人判緩刑 　韓國瑜下午將出面說明不用跳愛河！韓冰邀替高雄拚觀光　王世堅靦腆：可以廢物利用蔡英文要站第一線　林濁水諷：賴清德變隨身特助 

標題: 《他列民進黨10大豬隊友　「不下架將賠上2020」》
Sentence 1
 」前立委孫大千則說，「蔡總統又不能辭職下台，那要如何一肩扛起所有責任？
Sentence 2
 這不又是一句空話嗎？
Sentence 5
 第二，總統府秘書長陳菊理由：把高雄市整到「又老又窮」，和滿朝盡是「菊家班」。
